# Project: Build a Document Retriever Search Engine on Wikipedia Data

## Install OpenAI, and LangChain dependencies

In [1]:
!pip install langchain==0.3.11
!pip install langchain-openai==0.2.12
!pip install langchain-community==0.3.11

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.3/390.3 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.2 MB/s eta 0:00:00


## Install Chroma Vector DB and LangChain wrapper

In [2]:
!pip install langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.6/166.6 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 6.7 MB/s eta 0:00:

## Enter Open AI API Key

In [3]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

Enter Open AI API Key: ··········


## Setup Environment Variables

In [4]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY

### Open AI Embedding Models

LangChain enables us to access Open AI embedding models which include the newest models: a smaller and highly efficient `text-embedding-3-small` model, and a larger and more powerful `text-embedding-3-large` model.

In [5]:
from langchain_openai import OpenAIEmbeddings

# details here: https://openai.com/blog/new-embedding-models-and-api-updates
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

## Vector Databases

One of the most common ways to store and search over unstructured data is to embed it and store the resulting embedding vectors, and then at query time to embed the unstructured query and retrieve the embedding vectors that are 'most similar' to the embedded query. A vector database takes care of storing embedded data and performing vector search for you.

### Chroma Vector DB

[Chroma](https://docs.trychroma.com/getting-started) is a AI-native open-source vector database focused on developer productivity and happiness. Chroma is licensed under Apache 2.0.

### Get the wikipedia data

In [6]:
# if you can't download using the following code
# go to https://drive.google.com/file/d/1oWBnoxBZ1Mpeond8XDUSO6J9oAjcRDyW download it
# manually upload it on colab
!gdown 1oWBnoxBZ1Mpeond8XDUSO6J9oAjcRDyW

Downloading...
From (original): https://drive.google.com/uc?id=1oWBnoxBZ1Mpeond8XDUSO6J9oAjcRDyW
From (redirected): https://drive.google.com/uc?id=1oWBnoxBZ1Mpeond8XDUSO6J9oAjcRDyW&confirm=t&uuid=d49ca0f3-2dd7-46b2-a034-0103609a43b2
To: /content/simplewiki-2020-11-01.jsonl.gz
100% 50.2M/50.2M [00:00<00:00, 106MB/s]


In [7]:
import gzip
import json

wikipedia_filepath = 'simplewiki-2020-11-01.jsonl.gz'

docs = []
with gzip.open(wikipedia_filepath, 'rt', encoding='utf8') as fIn:
    for line in fIn:
        data = json.loads(line.strip())

        #Add all paragraphs
        #passages.extend(data['paragraphs'])

        #Only add the first paragraph
        docs.append({
                        'metadata': {
                                        'title': data.get('title'),
                                        'article_id': data.get('id')
                        },
                        'data': ' '.join(data.get('paragraphs')[0:3]) # restrict data to first 3 paragraphs to run later modules faster
        })

print("Passages:", len(docs))

Passages: 169597


In [8]:
# We subset our data so we only use a subset of wikipedia documents to run things faster
docs = [doc for doc in docs for x in ['linguistics', 'india', 'cheetah']
              if x in doc['data'].lower().split()]

In [9]:
len(docs)

1364

In [10]:
docs[:3]

[{'metadata': {'title': 'Kurgan hypothesis', 'article_id': '72554'},
  'data': 'The Kurgan model of Indo-European origins is about both the people and their Proto-Indo-European language. It uses both archaeology and linguistics to show the history of their culture at different stages of the Indo-European expansion. The Kurgan model is the most widely accepted theory on the origins of Indo-European.'},
 {'metadata': {'title': 'Marija Gimbutas', 'article_id': '72558'},
  'data': 'Marija Gimbutas (Lithuanian: Marija Gimbutienė, born Marija Birutė Alseikaitė) (Vilnius, Lithuania, January 23, 1921 – Los Angeles, United States February 2, 1994), was a Lithuanian-American archeologist, known for her research into the Neolithic and Bronze Age cultures of "Old Europe" and the theories that she introduced. Between 1946 and 1971, her writings merged traditional spadework with linguistics and mythologies.'},
 {'metadata': {'title': 'Basil', 'article_id': '73985'},
  'data': 'Basil ("Ocimum basilic

### Create LangChain Documents

In [11]:
from langchain.docstore.document import Document

docs = [Document(page_content=doc['data'],
                 metadata=doc['metadata']) for doc in docs]

In [12]:
docs[:3]

[Document(metadata={'title': 'Kurgan hypothesis', 'article_id': '72554'}, page_content='The Kurgan model of Indo-European origins is about both the people and their Proto-Indo-European language. It uses both archaeology and linguistics to show the history of their culture at different stages of the Indo-European expansion. The Kurgan model is the most widely accepted theory on the origins of Indo-European.'),
 Document(metadata={'title': 'Marija Gimbutas', 'article_id': '72558'}, page_content='Marija Gimbutas (Lithuanian: Marija Gimbutienė, born Marija Birutė Alseikaitė) (Vilnius, Lithuania, January 23, 1921 – Los Angeles, United States February 2, 1994), was a Lithuanian-American archeologist, known for her research into the Neolithic and Bronze Age cultures of "Old Europe" and the theories that she introduced. Between 1946 and 1971, her writings merged traditional spadework with linguistics and mythologies.'),
 Document(metadata={'title': 'Basil', 'article_id': '73985'}, page_content

In [13]:
len(docs)

1364

### Split larger documents into smaller chunks

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=300)
chunked_docs = splitter.split_documents(docs)

In [15]:
chunked_docs[:3]

[Document(metadata={'title': 'Kurgan hypothesis', 'article_id': '72554'}, page_content='The Kurgan model of Indo-European origins is about both the people and their Proto-Indo-European language. It uses both archaeology and linguistics to show the history of their culture at different stages of the Indo-European expansion. The Kurgan model is the most widely accepted theory on the origins of Indo-European.'),
 Document(metadata={'title': 'Marija Gimbutas', 'article_id': '72558'}, page_content='Marija Gimbutas (Lithuanian: Marija Gimbutienė, born Marija Birutė Alseikaitė) (Vilnius, Lithuania, January 23, 1921 – Los Angeles, United States February 2, 1994), was a Lithuanian-American archeologist, known for her research into the Neolithic and Bronze Age cultures of "Old Europe" and the theories that she introduced. Between 1946 and 1971, her writings merged traditional spadework with linguistics and mythologies.'),
 Document(metadata={'title': 'Basil', 'article_id': '73985'}, page_content

In [16]:
len(chunked_docs)

1388

### Create a Vector DB and persist on disk

Here we initialize a connection to a Chroma vector DB client, and also we want to save to disk, so we simply initialize the Chroma client and pass the directory where we want the data to be saved to.

In [17]:
from langchain_chroma import Chroma

# create vector DB of docs and embeddings - takes < 30s on Colab
chroma_db = Chroma.from_documents(documents=chunked_docs,
                                  collection_name='wikipedia_db',
                                  embedding=openai_embed_model,
                                  # need to set the distance function to cosine else it uses euclidean by default
                                  # check https://docs.trychroma.com/guides#changing-the-distance-function
                                  collection_metadata={"hnsw:space": "cosine"},
                                  persist_directory="./wikipedia_db")

### Load Vector DB from disk

This is just to show once you have a vector database on disk you can just load and create a connection to it anytime

In [18]:
# load from disk
chroma_db = Chroma(persist_directory="./wikipedia_db",
                   collection_name='wikipedia_db',
                   embedding_function=openai_embed_model)

In [19]:
chroma_db

## Experiment with Vector Database Retrievers

Here we will explore the following retrieval strategies on our Vector Database:

- Similarity or Ranking based Retrieval
- Multi Query Retrieval
- Contextual Compression Retrieval
- Chained Retrieval Pipeline

### Similarity or Ranking based Retrieval

We use cosine similarity here and retrieve the top 3 similar documents based on the user input query

In [20]:
similarity_retriever = chroma_db.as_retriever(search_type="similarity",
                                              search_kwargs={"k": 3})

In [21]:
query = "what is the capital of India?"
top3_docs = similarity_retriever.invoke(query)
top3_docs

[Document(metadata={'article_id': '5117', 'title': 'New Delhi'}, page_content='New Delhi () is the capital of India and a union territory of the megacity of Delhi. It has a very old history and is home to several monuments where the city is expensive to live in. In traditional Indian geography it falls under the North Indian zone. The city has an area of about 42.7\xa0km. New Delhi has a population of about 9.4 Million people.'),
 Document(metadata={'article_id': '5114', 'title': 'Mumbai'}, page_content="Mumbai (previously known as Bombay until 1996) is a natural harbor on the west coast of India, and is the capital city of Maharashtra state. It is India's largest city, and one of the world's most populous cities. It is the financial capital of India. The city is the second most-populous in the world. It has approximately 13 million people. Along with the neighboring cities of Navi Mumbai and Thane, it forms the world's 4th largest urban agglomeration. They have around 19.1 million peo

In [22]:
query = "what is the old capital of India?"
top3_docs = similarity_retriever.invoke(query)
top3_docs

[Document(metadata={'article_id': '5117', 'title': 'New Delhi'}, page_content='New Delhi () is the capital of India and a union territory of the megacity of Delhi. It has a very old history and is home to several monuments where the city is expensive to live in. In traditional Indian geography it falls under the North Indian zone. The city has an area of about 42.7\xa0km. New Delhi has a population of about 9.4 Million people.'),
 Document(metadata={'article_id': '22106', 'title': 'Delhi'}, page_content='Delhi (; "Dillī"; "Dillī"; "Dēhlī"), officially the National Capital Territory of Delhi (NCT), is a territory in India. It includes the country\'s capital New Delhi. It covers an area of . It is bigger than the Faroe Islands but smaller than Guadeloupe. Delhi is a part of the National Capital Region, which has 12.5 million residents. The governance of Delhi is like that of a state in India. It has its own legislature, high court and a council of executive ministers. Delhi is on the ban

In [23]:
query = "what is the fastest animal?"
top3_docs = similarity_retriever.invoke(query)
top3_docs

[Document(metadata={'article_id': '9800', 'title': 'Cheetah'}, page_content='A cheetah ("Acinonyx jubatus") is a medium large cat which lives in Africa. It is the fastest land animal and can run up to 112 kilometers per hour for a short time. Most cheetahs live in the savannas of Africa. There are a few in Asia. Cheetahs are active during the day, and hunt in the early morning or late evening. The cheetah compared to other big cats is light and slimly built. Its long thin legs and long spotted tail are necessary for fast running. Its lightly built, thin form is in sharp contrast with the robust build of other big cats. The head-and-body length ranges from . The cheetah stands 70 to 90\xa0cm at the shoulder, and weighs . The slightly curved claws are only weakly retractable (semi-retractable). This is a major point of difference between the cheetah and the other big cats, which have fully retractable claws.'),
 Document(metadata={'article_id': '538500', 'title': 'King cobra'}, page_cont

In [24]:
query = "what is the slowest animal?"
top3_docs = similarity_retriever.invoke(query)
top3_docs

[Document(metadata={'article_id': '558951', 'title': 'Slow loris'}, page_content='Slow lorises are the genus Nycticebus, nocturnal species of strepsirrhine primates. They live in southeast Asia and nearby areas. There are about eight species: the Sunda slow loris ("N.\xa0coucang"), Bengal slow loris ("N.\xa0bengalensis"), pygmy slow loris ("N.\xa0pygmaeus"), Javan slow loris ("N.\xa0javanicus"), Philippine slow loris ("N.\xa0menagensis"), Bangka slow loris ("N.\xa0bancanus"), Bornean slow loris ("N.\xa0borneanus"), and Kayan River slow loris ("N.\xa0kayan"). The group\'s closest relatives are the slender lorises of southern India and Sri Lanka. Their next closest relatives are the African lorisids, the pottos, false pottos, and angwantibos. They are less closely related to the remaining lorisoids (the various types of galago), and more distantly to the lemurs of Madagascar. Their evolutionary history is uncertain: their fossil record is patchy and molecular clock studies have given var

In [25]:
query = "what is the study of language?"
top3_docs = similarity_retriever.invoke(query)
top3_docs

[Document(metadata={'article_id': '20194', 'title': 'Linguistics'}, page_content='Linguistics is the study of language. People who study language are called linguists. There are five main parts of linguistics: the study of sounds (phonology), the study of parts of words, like "un-" and "-ing" (morphology), the study of word order and how sentences are made (syntax), the study of the meaning of words (semantics), and the study of the unspoken meaning of speech that is separate from the literal meaning of what is said (for example, saying "I\'m cold" to get someone to turn off the fan (pragmatics). There are many ways to use linguistics every day. Some linguists are theoretical linguists and study the theory and ideas behind language, such as historical linguistics (the study of the history of language, and how it has changed), or the study of how different groups of people may use language differently (sociolinguistics). Some linguists are applied linguists and use linguistics to do thi

### Multi Query Retrieval

Retrieval may produce different results with subtle changes in query wording, or if the embeddings do not capture the semantics of the data well. Prompt engineering / tuning is sometimes done to manually address these problems, but can be tedious.

The [`MultiQueryRetriever`](https://api.python.langchain.com/en/latest/retrievers/langchain.retrievers.multi_query.MultiQueryRetriever.html) automates the process of prompt tuning by using an LLM to generate multiple queries from different perspectives for a given user input query. For each query, it retrieves a set of relevant documents and takes the unique union across all queries to get a larger set of potentially relevant documents.

In [26]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [27]:
from langchain.retrievers.multi_query import MultiQueryRetriever
# Set logging for the queries
import logging

similarity_retriever = chroma_db.as_retriever(search_type="similarity",
                                              search_kwargs={"k": 3})

mq_retriever = MultiQueryRetriever.from_llm(
    retriever=similarity_retriever, llm=chatgpt
)

logging.basicConfig()
# so we can see what queries are generated by the LLM
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [28]:
query = "what is the capital of India?"
docs = mq_retriever.invoke(query)
docs

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you tell me the capital city of India?', '2. What city serves as the capital of India?', '3. Which city in India is designated as the capital?']


[Document(metadata={'article_id': '5117', 'title': 'New Delhi'}, page_content='New Delhi () is the capital of India and a union territory of the megacity of Delhi. It has a very old history and is home to several monuments where the city is expensive to live in. In traditional Indian geography it falls under the North Indian zone. The city has an area of about 42.7\xa0km. New Delhi has a population of about 9.4 Million people.'),
 Document(metadata={'article_id': '1968', 'title': 'Capital city'}, page_content="A capital city (or capital town or just capital) is a city or town, specified by law or constitution, by the government of a country, or part of a country, such as a state, province or county. It usually serves as the location of the government's central meeting place and offices. Most of the country's leaders and officials work in the capital city. Capitals are usually among the largest cities in their regions; often they are the biggest. For example, Montevideo is Uruguay's cap

In [29]:
query = "what is the old capital of India?"
docs = mq_retriever.invoke(query)
docs

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you tell me the former capital of India?', '2. What was the capital of India in the past?', '3. Which city used to be the capital of India?']


[Document(metadata={'article_id': '5117', 'title': 'New Delhi'}, page_content='New Delhi () is the capital of India and a union territory of the megacity of Delhi. It has a very old history and is home to several monuments where the city is expensive to live in. In traditional Indian geography it falls under the North Indian zone. The city has an area of about 42.7\xa0km. New Delhi has a population of about 9.4 Million people.'),
 Document(metadata={'article_id': '4062', 'title': 'Kolkata'}, page_content="Kolkata (spelled Calcutta before 1 January 2001) is the capital city of the Indian state of West Bengal. It is the second largest city in India after Mumbai. It is on the east bank of the River Hooghly. When it is called Calcutta, it includes the suburbs. This makes it the third largest city of India. This also makes it the world's 8th largest metropolitan area as defined by the United Nations. Kolkata served as the capital of India during the British Raj until 1911. Kolkata was once 

In [30]:
query = "what is the study of language?"
docs = mq_retriever.invoke(query)
docs

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you explain the field that focuses on the analysis of language?', '2. How would you define the academic discipline that examines language?', '3. What area of study is dedicated to exploring the structure and use of language?']


[Document(metadata={'article_id': '249472', 'title': 'Discourse analysis'}, page_content='Discourse analysis is a subject which studies a text or a conversation. This is a subject in linguistics which does not study sentences, like in syntax, but the entire text or conversation. The text or conversation is known as discourse. Discourse analyst prefer to use real life discourse in their studies, rather than invented sentences like in traditional linguistics. This way of studying real life discourse is called corpus linguistics. Discourse analysis is related to text linguistics. However, text linguistics studies how discourse is structured so that they are connected (how sentences are joined to each other). Discourse analysis studies this, and also how the discourse is connected to the context. This context includes who the people talking or writing are, the social and cultural context. Also, it studies the way mode, which is the way the language is represented (is it a letter, speech, e

### Contextual Compression Retrieval

The information most relevant to a query may be buried in a document with a lot of irrelevant text. Passing that full document through your application can lead to more expensive LLM calls and poorer responses.

Contextual compression is meant to fix this. The idea is simple: instead of immediately returning retrieved documents as-is, you can compress them using the context of the given query, so that only the relevant information is returned.

This compression can happen in the form of:

- Remove parts of the content of retrieved documents which are not relevant to the query. This is done by extracting only relevant parts of the document to the given query

- Filter out documents which are not relevant to the given query but do not remove content from the document

Here we wrap our multi-query retriever with a `ContextualCompressionRetriever`. Then we'll add an `LLMChainExtractor`, which will iterate over the initially returned documents and extract from each only the content that is relevant to the query.

In [31]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor


# extracts from each document only the content that is relevant to the query
compressor = LLMChainExtractor.from_llm(llm=chatgpt)

# retrieves the documents similar to query and then applies the compressor
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=mq_retriever
)

In [32]:
query = "what is the financial capital of India?"
docs = compression_retriever.invoke(query)
docs

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you tell me the economic hub of India?', '2. What city serves as the financial center of India?', '3. Which city in India is known for its financial activities?']


[Document(metadata={'article_id': '5114', 'title': 'Mumbai'}, page_content='Mumbai is the financial capital of India.'),
 Document(metadata={'article_id': '5117', 'title': 'New Delhi'}, page_content='New Delhi is the capital of India'),
 Document(metadata={'article_id': '351178', 'title': 'State Bank of India'}, page_content='Mumbai')]

In [33]:
query = "what is the old capital of India?"
docs = compression_retriever.invoke(query)
docs

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Which city served as the former capital of India?', '2. What was the historical capital of India?', '3. Can you tell me the previous capital city of India?']


[Document(metadata={'article_id': '4062', 'title': 'Kolkata'}, page_content='Calcutta became the capital of British India in 1772.'),
 Document(metadata={'article_id': '4062', 'title': 'Kolkata'}, page_content='Kolkata served as the capital of India during the British Raj until 1911.'),
 Document(metadata={'article_id': '5113', 'title': 'Chennai'}, page_content='Madras state')]

In [34]:
query = "what is the fastest animal?"
docs = compression_retriever.invoke(query)
docs

INFO:langchain.retrievers.multi_query:Generated queries: ['What animal holds the title for being the quickest?', 'Which creature is known for its incredible speed?', 'Can you tell me which animal is considered the fastest in the animal kingdom?']


[Document(metadata={'article_id': '9800', 'title': 'Cheetah'}, page_content='A cheetah ("Acinonyx jubatus") is a medium large cat which lives in Africa. It is the fastest land animal and can run up to 112 kilometers per hour for a short time.'),
 Document(metadata={'article_id': '528308', 'title': 'South African cheetah'}, page_content='The South African Cheetah ("Acinonyx jubatus jubatus")')]

In [35]:
query = "what is the slowest animal?"
docs = compression_retriever.invoke(query)
docs

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Which animal has the lowest speed?', '2. Can you tell me about the animal that moves the slowest?', '3. What is the least fast animal in the animal kingdom?']


[Document(metadata={'article_id': '558951', 'title': 'Slow loris'}, page_content='Slow lorises are the genus Nycticebus, nocturnal species of strepsirrhine primates.')]

In [36]:
query = "what is the study of language?"
docs = compression_retriever.invoke(query)
docs

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you explain the field that focuses on the analysis of language?', '2. How would you define the academic discipline that examines language?', '3. What area of study is dedicated to exploring the structure and use of language?']


[Document(metadata={'article_id': '249472', 'title': 'Discourse analysis'}, page_content='Discourse analysis is a subject which studies a text or a conversation. This is a subject in linguistics which does not study sentences, like in syntax, but the entire text or conversation. The text or conversation is known as discourse. Discourse analyst prefer to use real life discourse in their studies, rather than invented sentences like in traditional linguistics. This way of studying real life discourse is called corpus linguistics. Discourse analysis is related to text linguistics. However, text linguistics studies how discourse is structured so that they are connected (how sentences are joined to each other). Discourse analysis studies this, and also how the discourse is connected to the context. This context includes who the people talking or writing are, the social and cultural context. Also, it studies the way mode, which is the way the language is represented (is it a letter, speech, e

The `LLMChainFilter` is slightly simpler but more robust compressor that uses an LLM chain to decide which of the initially retrieved documents to filter out and which ones to return, without manipulating the document contents.

In [37]:
from langchain.retrievers.document_compressors import LLMChainFilter

#  decides which of the initially retrieved documents to filter out and which ones to return
_filter = LLMChainFilter.from_llm(llm=chatgpt)

# retrieves the documents similar to query and then applies the filter
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=mq_retriever
)

In [38]:
query = "what is the financial capital of India?"
docs = compression_retriever.invoke(query)
docs

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you tell me the economic hub of India?', '2. What city serves as the financial center of India?', '3. Which Indian city is known for its financial activities?']


[Document(metadata={'article_id': '5114', 'title': 'Mumbai'}, page_content="Mumbai (previously known as Bombay until 1996) is a natural harbor on the west coast of India, and is the capital city of Maharashtra state. It is India's largest city, and one of the world's most populous cities. It is the financial capital of India. The city is the second most-populous in the world. It has approximately 13 million people. Along with the neighboring cities of Navi Mumbai and Thane, it forms the world's 4th largest urban agglomeration. They have around 19.1 million people. The seven islands that form Bombay were home to fishing colonies. The islands were ruled by successive kingdoms and indigenous empires before Portuguese settlers took it. Then, it went to the British East India Company. During the mid-18th century, Bombay became a major trading town. It became a strong place for the Indian independence movement during the early 20th century. When India became independent in 1947, the city was

In [39]:
query = "what is the old capital of India?"
docs = compression_retriever.invoke(query)
docs

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Which city served as the former capital of India?', '2. What was the historical capital of India?', '3. Can you tell me the previous capital city of India?']


[Document(metadata={'article_id': '4062', 'title': 'Kolkata'}, page_content='forces started, in 1756 the British began to upgrade their fortifications. When this was protested, the Nawab of Bengal Siraj-Ud-Daulah attacked and captured Fort William. This led to the infamous Black Hole incident. A force of Company sepoys and British troops led by Robert Clive recaptured the city the next year. Calcutta became the capital of British India in 1772. However, the capital shifted to the hilly town of Shimla during the summer months every year, starting from the year 1864. Richard Wellesley, the Governor General between 1797–1805, helped in the growth of the city and its public architecture. This led to the description of Calcutta as "The City of Palaces". The city was a centre of the British East India Company\'s opium trade during the 18th and 19th century; locally produced opium was sold at auction in Kolkata, to be shipped to China.'),
 Document(metadata={'article_id': '4062', 'title': 'Ko

In [40]:
query = "what is the fastest animal?"
docs = compression_retriever.invoke(query)
docs

INFO:langchain.retrievers.multi_query:Generated queries: ['What animal holds the title for being the quickest?', 'Which creature is known for its incredible speed?', 'Can you tell me which animal is considered the fastest in the animal kingdom?']


[Document(metadata={'article_id': '9800', 'title': 'Cheetah'}, page_content='A cheetah ("Acinonyx jubatus") is a medium large cat which lives in Africa. It is the fastest land animal and can run up to 112 kilometers per hour for a short time. Most cheetahs live in the savannas of Africa. There are a few in Asia. Cheetahs are active during the day, and hunt in the early morning or late evening. The cheetah compared to other big cats is light and slimly built. Its long thin legs and long spotted tail are necessary for fast running. Its lightly built, thin form is in sharp contrast with the robust build of other big cats. The head-and-body length ranges from . The cheetah stands 70 to 90\xa0cm at the shoulder, and weighs . The slightly curved claws are only weakly retractable (semi-retractable). This is a major point of difference between the cheetah and the other big cats, which have fully retractable claws.'),
 Document(metadata={'article_id': '528308', 'title': 'South African cheetah'}

In [41]:
query = "what is the slowest animal?"
docs = compression_retriever.invoke(query)
docs

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Which animal has the lowest speed?', '2. Can you tell me about the animal that moves the slowest?', '3. What is the least fast animal in the animal kingdom?']


[Document(metadata={'article_id': '558951', 'title': 'Slow loris'}, page_content='Slow lorises are the genus Nycticebus, nocturnal species of strepsirrhine primates. They live in southeast Asia and nearby areas. There are about eight species: the Sunda slow loris ("N.\xa0coucang"), Bengal slow loris ("N.\xa0bengalensis"), pygmy slow loris ("N.\xa0pygmaeus"), Javan slow loris ("N.\xa0javanicus"), Philippine slow loris ("N.\xa0menagensis"), Bangka slow loris ("N.\xa0bancanus"), Bornean slow loris ("N.\xa0borneanus"), and Kayan River slow loris ("N.\xa0kayan"). The group\'s closest relatives are the slender lorises of southern India and Sri Lanka. Their next closest relatives are the African lorisids, the pottos, false pottos, and angwantibos. They are less closely related to the remaining lorisoids (the various types of galago), and more distantly to the lemurs of Madagascar. Their evolutionary history is uncertain: their fossil record is patchy and molecular clock studies have given var

### Chained Retrieval Pipeline

This strategy uses a chain of multiple retrievers sequentially to get to the most relevant documents. The following is the flow

Similarity Retrieval → Compression Filter → Reranker Model Retrieval

In [42]:
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker

# Retriever 1 - simple cosine distance based retriever
similarity_retriever = chroma_db.as_retriever(search_type="similarity",
                                              search_kwargs={"k": 5})

#  decides which of the initially retrieved documents to filter out and which ones to return
_filter = LLMChainFilter.from_llm(llm=chatgpt)
# Retriever 2 - retrieves the documents similar to query and then applies the filter
compressor_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=similarity_retriever
)

# download an open-source reranker model - BAAI/bge-reranker-v2-m3
reranker = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-base")
reranker_compressor = CrossEncoderReranker(model=reranker, top_n=3)
# Retriever 3 - Uses a Reranker model to rerank retrieval results from the previous retriever
final_retriever = ContextualCompressionRetriever(
    base_compressor=reranker_compressor, base_retriever=compressor_retriever
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [43]:
query = "what is the financial capital of India?"
docs = final_retriever.invoke(query)
docs

[Document(metadata={'article_id': '5114', 'title': 'Mumbai'}, page_content="Mumbai (previously known as Bombay until 1996) is a natural harbor on the west coast of India, and is the capital city of Maharashtra state. It is India's largest city, and one of the world's most populous cities. It is the financial capital of India. The city is the second most-populous in the world. It has approximately 13 million people. Along with the neighboring cities of Navi Mumbai and Thane, it forms the world's 4th largest urban agglomeration. They have around 19.1 million people. The seven islands that form Bombay were home to fishing colonies. The islands were ruled by successive kingdoms and indigenous empires before Portuguese settlers took it. Then, it went to the British East India Company. During the mid-18th century, Bombay became a major trading town. It became a strong place for the Indian independence movement during the early 20th century. When India became independent in 1947, the city was

In [44]:
query = "what is the old capital of India?"
docs = final_retriever.invoke(query)
docs

[Document(metadata={'article_id': '4062', 'title': 'Kolkata'}, page_content="Kolkata (spelled Calcutta before 1 January 2001) is the capital city of the Indian state of West Bengal. It is the second largest city in India after Mumbai. It is on the east bank of the River Hooghly. When it is called Calcutta, it includes the suburbs. This makes it the third largest city of India. This also makes it the world's 8th largest metropolitan area as defined by the United Nations. Kolkata served as the capital of India during the British Raj until 1911. Kolkata was once the center of industry and education. However, it has witnessed political violence and economic problems since 1954. Since 2000, Kolkata has grown due to economic growth. Like other metropolitan cities in India, Kolkata struggles with poverty, pollution and traffic congestion. The discovery of the nearby Chandraketugarh, an archaeological site has proved that people have lived there for over two millennia. The history of Kolkata b

In [45]:
query = "what is the fastest animal?"
docs = final_retriever.invoke(query)
docs

[Document(metadata={'article_id': '9800', 'title': 'Cheetah'}, page_content='A cheetah ("Acinonyx jubatus") is a medium large cat which lives in Africa. It is the fastest land animal and can run up to 112 kilometers per hour for a short time. Most cheetahs live in the savannas of Africa. There are a few in Asia. Cheetahs are active during the day, and hunt in the early morning or late evening. The cheetah compared to other big cats is light and slimly built. Its long thin legs and long spotted tail are necessary for fast running. Its lightly built, thin form is in sharp contrast with the robust build of other big cats. The head-and-body length ranges from . The cheetah stands 70 to 90\xa0cm at the shoulder, and weighs . The slightly curved claws are only weakly retractable (semi-retractable). This is a major point of difference between the cheetah and the other big cats, which have fully retractable claws.'),
 Document(metadata={'article_id': '528308', 'title': 'South African cheetah'}

In [46]:
query = "what is the slowest animal?"
docs = final_retriever.invoke(query)
docs

[Document(metadata={'article_id': '558951', 'title': 'Slow loris'}, page_content='Slow lorises are the genus Nycticebus, nocturnal species of strepsirrhine primates. They live in southeast Asia and nearby areas. There are about eight species: the Sunda slow loris ("N.\xa0coucang"), Bengal slow loris ("N.\xa0bengalensis"), pygmy slow loris ("N.\xa0pygmaeus"), Javan slow loris ("N.\xa0javanicus"), Philippine slow loris ("N.\xa0menagensis"), Bangka slow loris ("N.\xa0bancanus"), Bornean slow loris ("N.\xa0borneanus"), and Kayan River slow loris ("N.\xa0kayan"). The group\'s closest relatives are the slender lorises of southern India and Sri Lanka. Their next closest relatives are the African lorisids, the pottos, false pottos, and angwantibos. They are less closely related to the remaining lorisoids (the various types of galago), and more distantly to the lemurs of Madagascar. Their evolutionary history is uncertain: their fossil record is patchy and molecular clock studies have given var

In [47]:
query = "what is the study of language?"
docs = final_retriever.invoke(query)
docs

[Document(metadata={'article_id': '20194', 'title': 'Linguistics'}, page_content='Linguistics is the study of language. People who study language are called linguists. There are five main parts of linguistics: the study of sounds (phonology), the study of parts of words, like "un-" and "-ing" (morphology), the study of word order and how sentences are made (syntax), the study of the meaning of words (semantics), and the study of the unspoken meaning of speech that is separate from the literal meaning of what is said (for example, saying "I\'m cold" to get someone to turn off the fan (pragmatics). There are many ways to use linguistics every day. Some linguists are theoretical linguists and study the theory and ideas behind language, such as historical linguistics (the study of the history of language, and how it has changed), or the study of how different groups of people may use language differently (sociolinguistics). Some linguists are applied linguists and use linguistics to do thi

In [48]:
query = "what is chess?"
docs = final_retriever.invoke(query)
docs

[Document(metadata={'article_id': '65119', 'title': 'Shah'}, page_content='Shah () is a Persian word which means the king or ruler of a country. This word is used in different countries in the world, including Iran, India, Pakistan and Afghanistan. Presently the term "Shah" is used as a surname commonly by many people in India, Pakistan and Afghanistan who are Hindu, Moslem and Jain. Many Indian names which have "Shah" in them; the most famous of them is Shah Jahan, who as the Emperor of India ordered the creation of Taj Mahal. One possible etymology of the chess term "checkmate" is the Persian "shah mat", meaning "the king cannot escape" The term "Shah" often means Mohammad Reza Pahlavi, Shah of Iran from 1949 to 1979.'),
 Document(metadata={'article_id': '234589', 'title': 'History of chess'}, page_content='The history of chess goes back 2200 years. It was played in Chinese Warring Sates Period about 200BC, called XiangQi. Modern International Chess game though originated in northern